In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
import librosa
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import glob
from dotenv import load_dotenv
from pathlib import Path
import mysql.connector

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # INFO 숨김
from absl import logging as absl_logging
absl_logging.set_verbosity(absl_logging.ERROR)

In [76]:
def audio_preprocess(wav, sr=16000, n_mels=128):
  """input: 음성데이터 경로(str)
     output: Mel_spectrogram 변환데이터(np.array), Mel_spectrogram 길이(int)"""
  ori_y1, sr1 = librosa.load(wav, sr=sr)
  # mel_spec1 = librosa.feature.melspectrogram(y=ori_y1, sr=sr1, n_mels=n_mels, fmax=4200)
  mel_spec1 = librosa.feature.melspectrogram(y=ori_y1, sr=sr1, n_mels=n_mels)
  mel_db1 = librosa.power_to_db(mel_spec1, ref=np.max)
  # mel_db1 = mel_db1[:100,:]
  return mel_db1, mel_db1.shape[1]

In [77]:
# wav 데이터 padding
def wav_padding(wav, wav_max_len=312):
  """input: wav 데이터(np.array)
     output: padding된 wav 데이터(np.arrray)"""
  pad_width = wav_max_len - wav.shape[1]  # 얼마나 채워야 하는지
  if pad_width > 0:
      # 오른쪽(열 끝)에 0을 채움: ((행 시작, 행 끝), (열 시작, 열 끝))
      padded = np.pad(wav, pad_width=((0, 0), (0, pad_width)), mode='constant', constant_values=-80)
  elif pad_width == 0:
      padded = wav  # 이미 가장 김
  elif pad_width < 0:
      padded = wav[:, :wav_max_len]
  return padded

In [78]:
def x_data_preprocess(x, sr=16000, n_mels=128):
  temp_wav_list = []
  temp_wav_length_list = []
  if isinstance(x,(list,np.ndarray)):
    print("x_data_preprocess for문 시작")
    for path in tqdm(x):
      temp_wav_data, temp_wav_length = audio_preprocess(path,sr,n_mels)
      temp_pad_wav_data = wav_padding(temp_wav_data)
      temp_wav_list.append(temp_pad_wav_data)
      temp_wav_length_list.append(temp_wav_length)

    temp_x_padded_data = np.array(temp_wav_list)
    temp_x_data = np.transpose(temp_x_padded_data, (0, 2, 1))
    temp_x_data_length = np.array(temp_wav_length_list)

    return temp_x_data, temp_x_data_length
  elif isinstance(x,str):
    temp_wav_data, temp_x_data_length = audio_preprocess(x,sr,n_mels)
    temp_x_data = wav_padding(temp_wav_data)
    return temp_x_data, temp_x_data_length

In [79]:
def pred_preprocess(wav_path, sr=16000, n_mels=80):
  pred_wav = wav_path
  pred_,_,_ = audio_preprocess(pred_wav,sr,n_mels)
  padd_pred = wav_padding(pred_)
  x_padded_pred_data = np.stack([padd_pred])
  pred_audio_transposed = np.transpose(x_padded_pred_data, (0, 2, 1))
  x_pred_data = np.expand_dims(pred_audio_transposed, axis=-1)
  return x_pred_data

In [80]:
def build_ptk_model(model_name=None):
  inputs = tf.keras.layers.Input(shape=(312,128,1),name="x_input")

  BatNor_01 = tf.keras.layers.BatchNormalization(momentum=0.9)
  BatNor_01_out = BatNor_01(inputs)

  Conv2d_01 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')
  Conv2d_01_out = Conv2d_01(BatNor_01_out)
  Conv2d_02 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')
  Conv2d_02_out = Conv2d_02(Conv2d_01_out)
  AvgPool_01 = tf.keras.layers.AveragePooling2D((2, 2))
  AvgPool_01_out = AvgPool_01(Conv2d_02_out)

  Conv2d_03 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')
  Conv2d_03_out = Conv2d_03(AvgPool_01_out)
  Conv2d_04 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')
  Conv2d_04_out = Conv2d_04(Conv2d_03_out)
  AvgPool_02 = tf.keras.layers.AveragePooling2D((2, 2))
  AvgPool_02_out = AvgPool_02(Conv2d_04_out)

  Conv2d_05 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')
  Conv2d_05_out = Conv2d_05(AvgPool_02_out)
  Conv2d_06 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')
  Conv2d_06_out = Conv2d_06(Conv2d_05_out)
  AvgPool_03 = tf.keras.layers.AveragePooling2D((2, 2))
  AvgPool_03_out = AvgPool_03(Conv2d_06_out)

  b,t,f,c = AvgPool_03_out.shape
  Reshape_01 = tf.keras.layers.Reshape((-1,f*c))
  Reshape_01_out = Reshape_01(AvgPool_03_out)

  Dense_01 = tf.keras.layers.Dense(128, activation='relu')
  Dense_01_out = Dense_01(Reshape_01_out)

  Flat_01 = tf.keras.layers.Flatten()
  Flat_01_out = Flat_01(Dense_01_out)

  Dense_02 = tf.keras.layers.Dense(256, activation='relu')
  Dense_02_out = Dense_02(Flat_01_out)

  Dense_03 = tf.keras.layers.Dense(1)
  Dense_03_out = Dense_03(Dense_02_out)

  if model_name == None:
    return Model(inputs=inputs, outputs=Dense_03_out)
  else:
    return Model(inputs=inputs, outputs=Dense_03_out, name=f'{model_name}')

In [ ]:
win_ip = '172.26.80.1'

In [ ]:
env_path = "/home/inter/project/ptk/f.env"
load_dotenv(dotenv_path=env_path)

def get_connection():
    host_env = os.getenv("db_host", "")
    host = win_ip if host_env in ("", "localhost", "127.0.0.1") else host_env

    conn = mysql.connector.connect(
        host=host,
        port=int(os.getenv("db_port", "3306")),
        database=os.getenv("db_database"),
        user=os.getenv("db_username"),
        password=os.getenv("db_password"),
    )
    return conn

def is_invalid(value):
    return pd.isna(value) or value == ""

# ####################################### #
# get_file_lst
# 파일 경로와 목록 정보를 조회
# ####################################### #
def get_file_lst(assess_type, question_cd, question_no=None, order_num=None):
    conn = None
    cursor = None
    try:
        conn = get_connection()
        cursor = conn.cursor()

        sql = ""
        sql +=  "select lst.ORDER_NUM, lst.ASSESS_TYPE, lst.QUESTION_CD, lst.QUESTION_NO, lst.QUESTION_MINOR_NO "
        sql +=  "   , concat(lst.MAIN_PATH,'/',lst.SUB_PATH) as PATH, lst.FILE_NAME, lst.DURATION, lst.RATE "
        sql +=  "	, ref.SCORE, alc.SCORE_ALLOCATION as alc_score, alc.note "
        sql +=  "from assess_file_lst lst "
        sql +=  "	inner join assess_lst alst "
        sql +=  "		on lst.PATIENT_ID = alst.PATIENT_ID "
        sql +=  "		and lst.ORDER_NUM = alst.ORDER_NUM "
        sql +=  "		and alst.EXCLUDED = '0' "
        sql +=  "	inner join assess_score_reference ref "
        sql +=  "		on lst.PATIENT_ID = ref.PATIENT_ID "
        sql +=  "		and lst.ORDER_NUM = ref.ORDER_NUM "
        sql +=  "		and lst.ASSESS_TYPE = ref.ASSESS_TYPE "
        sql +=  "		and lst.QUESTION_CD = ref.QUESTION_CD "
        sql +=  "		and lst.QUESTION_NO = ref.QUESTION_NO "
        sql +=  "       and ref.USE_YN = 'Y' "
        sql +=  "	left outer join assess_score_allocation alc "
        sql +=  "		on lst.ASSESS_TYPE = alc.ASSESS_TYPE "
        sql +=  "		and lst.QUESTION_CD = alc.QUESTION_CD "
        sql +=  "		and lst.QUESTION_NO = alc.QUESTION_NO "
        sql += f"where lst.ASSESS_TYPE = '{assess_type}' "
        sql += f"and lst.QUESTION_CD = '{question_cd}' "
        if question_no:
            sql += f"and lst.QUESTION_NO = {question_no} "
        if order_num:
            sql += f"and lst.ORDER_NUM = {order_num} "
        sql +=  "and lst.USE_YN = 'Y' "
    
        cursor.execute(sql)
        rows = cursor.fetchall()
        ret_df = pd.DataFrame(rows, columns=['ORDER_NUM', 'ASSESS_TYPE', 'QUESTION_CD', 'QUESTION_NO', 'QUESTION_MINOR_NO', 'Path','File Name','Duration', 'Rate', 'Score(Refer)', 'Score(Alloc)', 'Note' ])

        msg = f'{len(ret_df)}건의 데이터가 조회되었습니다.'
        return msg, ret_df

    except Exception as e:
        return f"오류 발생: {str(e)}", None

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [83]:
_,ptk_sound = get_file_lst("CLAP_D","PTK_SOUND")
ptk_sound.drop(columns=["ORDER_NUM","QUESTION_CD","ASSESS_TYPE","QUESTION_MINOR_NO","Duration","Rate","Score(Alloc)","Note"],inplace=True)
ptk_sound.columns = ['QUESTION_NO', 'Path', 'File Name', 'Target']

folder_path = "upload"

for i in range(len(ptk_sound)):
    temp_path = ptk_sound.loc[i,'Path']
    file_name = ptk_sound.loc[i,'File Name']
    ptk_sound.loc[i,'P_Num'] = temp_path.split('/')[0]
    ptk_sound.loc[i,'New_Path'] = f"{folder_path}/{temp_path.upper().replace('-','_')}/{file_name}"

ptk_sound

,QUESTION_NO,Path,File Name,Target,P_Num,New_Path
0,1,1001/CLAP_D/1,p_1_0.wav,3.8,1001,upload/1001/CLAP_D/1/p_1_0.wav
1,2,1001/CLAP_D/1,p_2_0.wav,4.0,1001,upload/1001/CLAP_D/1/p_2_0.wav
2,3,1001/CLAP_D/1,p_3_0.wav,2.0,1001,upload/1001/CLAP_D/1/p_3_0.wav
3,4,1001/CLAP_D/1,p_4_0.wav,4.4,1001,upload/1001/CLAP_D/1/p_4_0.wav
4,5,1001/CLAP_D/1,p_5_0.wav,3.8,1001,upload/1001/CLAP_D/1/p_5_0.wav
...,...,...,...,...,...,...
713,8,5067/CLAP_D/1,p_8_0.wav,2.4,5067,upload/5067/CLAP_D/1/p_8_0.wav
714,9,5067/CLAP_D/1,p_9_0.wav,2.2,5067,upload/5067/CLAP_D/1/p_9_0.wav
715,10,5067/CLAP_D/1,p_10_0.wav,0.8,5067,upload/5067/CLAP_D/1/p_10_0.wav
716,11,5067/CLAP_D/1,p_11_0.wav,0.8,5067,upload/5067/CLAP_D/1/p_11_0.wav


In [99]:
peo_wav_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_1_')|
                          ptk_sound['File Name'].str.contains('p_2_')|
                          ptk_sound['File Name'].str.contains('p_3_')),"New_Path"].to_list()
peo_tar_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_1_')|
                          ptk_sound['File Name'].str.contains('p_2_')|
                          ptk_sound['File Name'].str.contains('p_3_')),"Target"].to_list()

teo_wav_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_4_')|
                          ptk_sound['File Name'].str.contains('p_5_')|
                          ptk_sound['File Name'].str.contains('p_6_')),"New_Path"].to_list()
teo_tar_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_4_')|
                          ptk_sound['File Name'].str.contains('p_5_')|
                          ptk_sound['File Name'].str.contains('p_6_')),"Target"].to_list()

keo_wav_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_7_')|
                          ptk_sound['File Name'].str.contains('p_8_')|
                          ptk_sound['File Name'].str.contains('p_9_')),"New_Path"].to_list()
keo_tar_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_7_')|
                          ptk_sound['File Name'].str.contains('p_8_')|
                          ptk_sound['File Name'].str.contains('p_9_')),"Target"].to_list()

ptk_con_wav_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_1_')|
                          ptk_sound['File Name'].str.contains('p_2_')|
                          ptk_sound['File Name'].str.contains('p_3_')|
                          ptk_sound['File Name'].str.contains('p_4_')|
                          ptk_sound['File Name'].str.contains('p_5_')|
                          ptk_sound['File Name'].str.contains('p_6_')|
                          ptk_sound['File Name'].str.contains('p_7_')|
                          ptk_sound['File Name'].str.contains('p_8_')|
                          ptk_sound['File Name'].str.contains('p_9_')),"New_Path"].to_list()
ptk_con_tar_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_1_')|
                          ptk_sound['File Name'].str.contains('p_2_')|
                          ptk_sound['File Name'].str.contains('p_3_')|
                          ptk_sound['File Name'].str.contains('p_4_')|
                          ptk_sound['File Name'].str.contains('p_5_')|
                          ptk_sound['File Name'].str.contains('p_6_')|
                          ptk_sound['File Name'].str.contains('p_7_')|
                          ptk_sound['File Name'].str.contains('p_8_')|
                          ptk_sound['File Name'].str.contains('p_9_')),"Target"].to_list()

ptk_wav_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_10_')|
                          ptk_sound['File Name'].str.contains('p_11_')|
                          ptk_sound['File Name'].str.contains('p_12_')),"New_Path"].to_list()
ptk_tar_list = ptk_sound.loc[(ptk_sound['File Name'].str.contains('p_10_')|
                          ptk_sound['File Name'].str.contains('p_11_')|
                          ptk_sound['File Name'].str.contains('p_12_')),"Target"].to_list()

In [100]:
peo_x_data, peo_x_data_length = x_data_preprocess(peo_wav_list)
teo_x_data, teo_x_data_length = x_data_preprocess(teo_wav_list)
keo_x_data, keo_x_data_length = x_data_preprocess(keo_wav_list)
ptk_con_x_data, ptk_con_x_data_length = x_data_preprocess(ptk_con_wav_list)
ptk_x_data, ptk_x_data_length = x_data_preprocess(ptk_wav_list)

x_data_preprocess for문 시작


100%|██████████| 180/180 [00:01<00:00, 144.20it/s]


x_data_preprocess for문 시작


100%|██████████| 180/180 [00:01<00:00, 148.81it/s]


x_data_preprocess for문 시작


100%|██████████| 180/180 [00:01<00:00, 148.58it/s]


x_data_preprocess for문 시작


100%|██████████| 540/540 [00:03<00:00, 135.68it/s]


x_data_preprocess for문 시작


100%|██████████| 178/178 [00:01<00:00, 148.70it/s]


In [101]:
peo_y_data = np.array(peo_tar_list)
teo_y_data = np.array(teo_tar_list)
keo_y_data = np.array(keo_tar_list)
ptk_con_y_data = np.array(ptk_con_tar_list)
ptk_y_data = np.array(ptk_tar_list)

peo_y_data = peo_y_data.astype(np.float32)
teo_y_data = teo_y_data.astype(np.float32)
keo_y_data = keo_y_data.astype(np.float32)
ptk_con_y_data = ptk_con_y_data.astype(np.float32)
ptk_y_data = ptk_y_data.astype(np.float32)

In [ ]:
peo_x_train_, peo_x_test, peo_x_data_length_train_, peo_x_data_length_test, peo_y_train_, peo_y_test = train_test_split(peo_x_data, peo_x_data_length, peo_y_data, test_size=0.1)
peo_x_train, peo_x_valid, peo_x_data_length_train, peo_x_data_length_valid, peo_y_train, peo_y_valid = train_test_split(peo_x_train_, peo_x_data_length_train_, peo_y_train_, test_size=0.1)

# peo_x_train.shape, peo_x_valid.shape, peo_x_test.shape, peo_x_data_length_train.shape, peo_x_data_length_valid.shape, peo_x_data_length_test.shape, peo_y_train.shape, peo_y_valid.shape, peo_y_test.shape

In [88]:
teo_x_train_, teo_x_test, teo_x_data_length_train_, teo_x_data_length_test, teo_y_train_, teo_y_test = train_test_split(teo_x_data, teo_x_data_length, teo_y_data, test_size=0.1)
teo_x_train, teo_x_valid, teo_x_data_length_train, teo_x_data_length_valid, teo_y_train, teo_y_valid = train_test_split(teo_x_train_, teo_x_data_length_train_, teo_y_train_, test_size=0.1)

# teo_x_train.shape, teo_x_valid.shape, teo_x_test.shape, teo_x_data_length_train.shape, teo_x_data_length_valid.shape, teo_x_data_length_test.shape, teo_y_train.shape, teo_y_valid.shape, teo_y_test.shape

In [89]:
keo_x_train_, keo_x_test, keo_x_data_length_train_, keo_x_data_length_test, keo_y_train_, keo_y_test = train_test_split(keo_x_data, keo_x_data_length, keo_y_data, test_size=0.1)
keo_x_train, keo_x_valid, keo_x_data_length_train, keo_x_data_length_valid, keo_y_train, keo_y_valid = train_test_split(keo_x_train_, keo_x_data_length_train_, keo_y_train_, test_size=0.1)

# keo_x_train.shape, keo_x_valid.shape, keo_x_test.shape, keo_x_data_length_train.shape, keo_x_data_length_valid.shape, keo_x_data_length_test.shape, keo_y_train.shape, keo_y_valid.shape, keo_y_test.shape

In [102]:
ptk_con_x_train_, ptk_con_x_test, ptk_con_x_data_length_train_, ptk_con_x_data_length_test, ptk_con_y_train_, ptk_con_y_test = train_test_split(ptk_con_x_data, ptk_con_x_data_length, ptk_con_y_data, test_size=0.1)
ptk_con_x_train, ptk_con_x_valid, ptk_con_x_data_length_train, ptk_con_x_data_length_valid, ptk_con_y_train, ptk_con_y_valid = train_test_split(ptk_con_x_train_, ptk_con_x_data_length_train_, ptk_con_y_train_, test_size=0.1)

# ptk_con_x_train.shape, ptk_con_x_valid.shape, ptk_con_x_test.shape, ptk_con_x_data_length_train.shape, ptk_con_x_data_length_valid.shape, ptk_con_x_data_length_test.shape, ptk_con_y_train.shape, ptk_con_y_valid.shape, ptk_con_y_test.shape

In [90]:
ptk_x_train_, ptk_x_test, ptk_x_data_length_train_, ptk_x_data_length_test, ptk_y_train_, ptk_y_test = train_test_split(ptk_x_data, ptk_x_data_length, ptk_y_data, test_size=0.1)
ptk_x_train, ptk_x_valid, ptk_x_data_length_train, ptk_x_data_length_valid, ptk_y_train, ptk_y_valid = train_test_split(ptk_x_train_, ptk_x_data_length_train_, ptk_y_train_, test_size=0.1)

# ptk_x_train.shape, ptk_x_valid.shape, ptk_x_test.shape, ptk_x_data_length_train.shape, ptk_x_data_length_valid.shape, ptk_x_data_length_test.shape, ptk_y_train.shape, ptk_y_valid.shape, ptk_y_test.shape

In [91]:
peo_model_1 = build_ptk_model()
peo_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

early = EarlyStopping(patience=10, monitor='val_loss')
model_chk = ModelCheckpoint('peo_model.keras',
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=3)

peo_train_history = peo_model_1.fit(x=peo_x_train,
                                    y=peo_y_train,
                                    epochs=100,
                                    batch_size=32,
                                    validation_data=(peo_x_valid, peo_y_valid),
                                    callbacks=[early,model_chk]
                                    )

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step - loss: 10.9336 - mae: 2.9542
Epoch 1: val_loss improved from None to 4.01934, saving model to peo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 637ms/step - loss: 8.5291 - mae: 2.5613 - val_loss: 4.0193 - val_mae: 1.7774
Epoch 2/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 6.9089 - mae: 2.2402
Epoch 2: val_loss did not improve from 4.01934
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 4.9663 - mae: 1.8266 - val_loss: 8.6111 - val_mae: 2.6497
Epoch 3/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 4.1095 - mae: 1.5476
Epoch 3: val_loss improved from 4.01934 to 3.22895, saving model to peo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 4.1597 - mae: 1.6353 - val_loss: 3.2290 - val_mae: 1.6010
Epoch 4/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.0243 - mae: 1.4977
Epoch 4: val_loss improved from 3.22895 to 1.47418, saving model to peo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 3.0587 - mae: 1.4166 - val_

In [92]:
teo_model_1 = build_ptk_model()
teo_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

early = EarlyStopping(patience=10, monitor='val_loss')
model_chk = ModelCheckpoint('teo_model.keras',
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=3)

teo_train_history = teo_model_1.fit(x=teo_x_train,
                                    y=teo_y_train,
                                    epochs=100,
                                    batch_size=32,
                                    validation_data=(teo_x_valid, teo_y_valid),
                                    callbacks=[early,model_chk]
                                    )

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - loss: 9.8632 - mae: 2.7119
Epoch 1: val_loss improved from None to 38.55782, saving model to teo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 614ms/step - loss: 7.3020 - mae: 2.2208 - val_loss: 38.5578 - val_mae: 5.7994
Epoch 2/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 1.9801 - mae: 1.1548
Epoch 2: val_loss improved from 38.55782 to 2.84836, saving model to teo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 1.8831 - mae: 1.1225 - val_loss: 2.8484 - val_mae: 1.4756
Epoch 3/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 1.9737 - mae: 1.1591
Epoch 3: val_loss did not improve from 2.84836
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 2.1155 - mae: 1.1633 - val_loss: 4.0741 - val_mae: 1.8596
Epoch 4/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 1.8693 - mae: 1.1118
Epoch 4: val_loss did not improve from 2.84836
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 2.0337 - mae: 1.1387 - val_loss: 4.1803 - val_mae: 1.8838
Epo

In [93]:
keo_model_1 = build_ptk_model()
keo_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

early = EarlyStopping(patience=10, monitor='val_loss')
model_chk = ModelCheckpoint('keo_model.keras',
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=3)

keo_train_history = keo_model_1.fit(x=keo_x_train,
                                    y=keo_y_train,
                                    epochs=100,
                                    batch_size=32,
                                    validation_data=(keo_x_valid, keo_y_valid),
                                    callbacks=[early,model_chk]
                                    )

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step - loss: 7.8527 - mae: 2.4296
Epoch 1: val_loss improved from None to 59.82996, saving model to keo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 606ms/step - loss: 6.0454 - mae: 2.0765 - val_loss: 59.8300 - val_mae: 7.3857
Epoch 2/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 3.2980 - mae: 1.3932
Epoch 2: val_loss improved from 59.82996 to 4.67074, saving model to keo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 4.1846 - mae: 1.6507 - val_loss: 4.6707 - val_mae: 1.7621
Epoch 3/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 4.3750 - mae: 1.8320
Epoch 3: val_loss did not improve from 4.67074
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 3.4297 - mae: 1.5438 - val_loss: 7.4873 - val_mae: 2.3046
Epoch 4/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 3.3862 - mae: 1.4515
Epoch 4: val_loss improved from 4.67074 to 3.31243, saving model to keo_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 2.5079 - mae: 1.2599 - v

In [138]:
ptk_con_model_1 = build_ptk_model()
ptk_con_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

early = EarlyStopping(patience=10, monitor='val_loss')
model_chk = ModelCheckpoint('ptk_con_model.keras',
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=3)

ptk_con_train_history = ptk_con_model_1.fit(x=ptk_con_x_train,
                                    y=ptk_con_y_train,
                                    epochs=100,
                                    batch_size=32,
                                    validation_data=(ptk_con_x_valid, ptk_con_y_valid),
                                    callbacks=[early,model_chk]
                                    )

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - loss: 15.8810 - mae: 3.4823
Epoch 1: val_loss improved from None to 9.14686, saving model to ptk_con_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 310ms/step - loss: 14.4246 - mae: 3.3333 - val_loss: 9.1469 - val_mae: 2.6642
Epoch 2/100
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 6.9885 - mae: 2.2106
Epoch 2: val_loss improved from 9.14686 to 2.32861, saving model to ptk_con_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 4.8021 - mae: 1.8106 - val_loss: 2.3286 - val_mae: 1.2908
Epoch 3/100
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2.9565 - mae: 1.3857
Epoch 3: val_loss improved from 2.32861 to 1.78711, saving model to ptk_con_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 2.8677 - mae: 1.3699 - val_loss: 1.7871 - val_mae: 1.1499
Epoch 4/100
13/14 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2.2572 - mae: 1.2048
Epoch 4: val_loss improved from 1.78711 to 1.43515, saving model to ptk_con_model.keras
14/14

In [140]:
ptk_model_1 = build_ptk_model()
ptk_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='mse', metrics=['mae'])

early = EarlyStopping(patience=10, monitor='val_loss')
model_chk = ModelCheckpoint('ptk_model.keras',
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=3)

ptk_train_history = ptk_model_1.fit(x=ptk_x_train,
                                    y=ptk_y_train,
                                    epochs=100,
                                    batch_size=32,
                                    validation_data=(ptk_x_valid, ptk_y_valid),
                                    callbacks=[early,model_chk]
                                    )

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step - loss: 2.0979 - mae: 1.2185
Epoch 1: val_loss improved from None to 0.87369, saving model to ptk_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 574ms/step - loss: 2.0562 - mae: 1.1845 - val_loss: 0.8737 - val_mae: 0.7265
Epoch 2/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 1.7032 - mae: 1.1055
Epoch 2: val_loss improved from 0.87369 to 0.78654, saving model to ptk_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - loss: 1.7584 - mae: 1.1357 - val_loss: 0.7865 - val_mae: 0.6988
Epoch 3/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 1.1604 - mae: 0.9061
Epoch 3: val_loss did not improve from 0.78654
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.9776 - mae: 0.8179 - val_loss: 1.0555 - val_mae: 0.8856
Epoch 4/100
4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.5708 - mae: 0.6195
Epoch 4: val_loss improved from 0.78654 to 0.55273, saving model to ptk_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.6662 - mae: 0.6718 - val_

In [144]:
peo_peo_pred = peo_model_1.predict(peo_x_test)
peo_teo_pred = peo_model_1.predict(teo_x_test)
peo_keo_pred = peo_model_1.predict(keo_x_test)
peo_ptk_con_pred = peo_model_1.predict(ptk_con_x_test)
peo_ptk_pred = peo_model_1.predict(ptk_x_test)


teo_peo_pred = teo_model_1.predict(peo_x_test)
teo_teo_pred = teo_model_1.predict(teo_x_test)
teo_keo_pred = teo_model_1.predict(keo_x_test)
teo_ptk_con_pred = teo_model_1.predict(ptk_con_x_test)
teo_ptk_pred = teo_model_1.predict(ptk_x_test)

keo_peo_pred = keo_model_1.predict(peo_x_test)
keo_teo_pred = keo_model_1.predict(teo_x_test)
keo_keo_pred = keo_model_1.predict(keo_x_test)
keo_ptk_con_pred = keo_model_1.predict(ptk_con_x_test)
keo_ptk_pred = keo_model_1.predict(ptk_x_test)

ptk_con_peo_pred = ptk_con_model_1.predict(peo_x_test)
ptk_con_teo_pred = ptk_con_model_1.predict(teo_x_test)
ptk_con_keo_pred = ptk_con_model_1.predict(keo_x_test)
ptk_con_ptk_con_pred = ptk_con_model_1.predict(ptk_con_x_test)
ptk_con_ptk_pred = ptk_con_model_1.predict(ptk_x_test)

ptk_peo_pred = ptk_model_1.predict(peo_x_test)
ptk_teo_pred = ptk_model_1.predict(teo_x_test)
ptk_keo_pred = ptk_model_1.predict(keo_x_test)
ptk_ptk_con_pred = ptk_model_1.predict(ptk_con_x_test)
ptk_ptk_pred = ptk_model_1.predict(ptk_x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [145]:
peo_peo_pred = peo_peo_pred.reshape(18)
peo_teo_pred = peo_teo_pred.reshape(18)
peo_keo_pred = peo_keo_pred.reshape(18)
peo_ptk_con_pred = peo_ptk_con_pred.reshape(54)
peo_ptk_pred = peo_ptk_pred.reshape(18)

teo_peo_pred = teo_peo_pred.reshape(18)
teo_teo_pred = teo_teo_pred.reshape(18)
teo_keo_pred = teo_keo_pred.reshape(18)
teo_ptk_con_pred = teo_ptk_con_pred.reshape(54)
teo_ptk_pred = teo_ptk_pred.reshape(18)


keo_peo_pred = keo_peo_pred.reshape(18)
keo_teo_pred = keo_teo_pred.reshape(18)
keo_keo_pred = keo_keo_pred.reshape(18)
keo_ptk_con_pred = keo_ptk_con_pred.reshape(54)
keo_ptk_pred = keo_ptk_pred.reshape(18)

ptk_con_peo_pred = ptk_con_peo_pred.reshape(18)
ptk_con_teo_pred = ptk_con_teo_pred.reshape(18)
ptk_con_keo_pred = ptk_con_keo_pred.reshape(18)
ptk_con_ptk_con_pred = ptk_con_ptk_con_pred.reshape(54)
ptk_con_ptk_pred = ptk_con_ptk_pred.reshape(18)

ptk_peo_pred = ptk_peo_pred.reshape(18)
ptk_teo_pred = ptk_teo_pred.reshape(18)
ptk_keo_pred = ptk_keo_pred.reshape(18)
ptk_ptk_con_pred = ptk_ptk_con_pred.reshape(54)
ptk_ptk_pred = ptk_ptk_pred.reshape(18)

In [146]:
np.corrcoef(peo_peo_pred, peo_y_test), np.corrcoef(peo_teo_pred, teo_y_test), np.corrcoef(peo_keo_pred, keo_y_test), np.corrcoef(peo_ptk_con_pred, ptk_con_y_test), np.corrcoef(peo_ptk_pred, ptk_y_test)

(array([[1.        , 0.68070265],
        [0.68070265, 1.        ]]),
 array([[1.        , 0.89268122],
        [0.89268122, 1.        ]]),
 array([[1.       , 0.8603269],
        [0.8603269, 1.       ]]),
 array([[1.        , 0.88792873],
        [0.88792873, 1.        ]]),
 array([[1.        , 0.86444198],
        [0.86444198, 1.        ]]))

In [147]:
np.corrcoef(teo_peo_pred, peo_y_test), np.corrcoef(teo_teo_pred, teo_y_test), np.corrcoef(teo_keo_pred, keo_y_test), np.corrcoef(teo_ptk_con_pred, ptk_con_y_test), np.corrcoef(teo_ptk_pred, ptk_y_test)

(array([[1.        , 0.70062426],
        [0.70062426, 1.        ]]),
 array([[1.       , 0.8692531],
        [0.8692531, 1.       ]]),
 array([[1.        , 0.63560769],
        [0.63560769, 1.        ]]),
 array([[1.        , 0.90212558],
        [0.90212558, 1.        ]]),
 array([[1.        , 0.81967613],
        [0.81967613, 1.        ]]))

In [148]:
np.corrcoef(keo_peo_pred, peo_y_test), np.corrcoef(keo_teo_pred, teo_y_test), np.corrcoef(keo_keo_pred, keo_y_test), np.corrcoef(keo_ptk_con_pred, ptk_con_y_test), np.corrcoef(keo_ptk_pred, ptk_y_test)

(array([[1.        , 0.63345277],
        [0.63345277, 1.        ]]),
 array([[1.        , 0.89127556],
        [0.89127556, 1.        ]]),
 array([[1.        , 0.86503685],
        [0.86503685, 1.        ]]),
 array([[1.        , 0.86158824],
        [0.86158824, 1.        ]]),
 array([[1.        , 0.87882713],
        [0.87882713, 1.        ]]))

In [150]:
np.corrcoef(ptk_con_peo_pred, peo_y_test), np.corrcoef(ptk_con_teo_pred, teo_y_test), np.corrcoef(ptk_con_keo_pred, keo_y_test), np.corrcoef(ptk_con_ptk_con_pred, ptk_con_y_test), np.corrcoef(ptk_con_ptk_pred, ptk_y_test)

(array([[1.        , 0.95895055],
        [0.95895055, 1.        ]]),
 array([[1.        , 0.97924709],
        [0.97924709, 1.        ]]),
 array([[1.        , 0.95049107],
        [0.95049107, 1.        ]]),
 array([[1.        , 0.81220161],
        [0.81220161, 1.        ]]),
 array([[1.        , 0.83227535],
        [0.83227535, 1.        ]]))

In [151]:
np.corrcoef(ptk_peo_pred, peo_y_test), np.corrcoef(ptk_teo_pred, teo_y_test), np.corrcoef(ptk_keo_pred, keo_y_test), np.corrcoef(ptk_ptk_con_pred, ptk_con_y_test), np.corrcoef(ptk_ptk_pred, ptk_y_test)

(array([[1.        , 0.52292429],
        [0.52292429, 1.        ]]),
 array([[1.        , 0.89443128],
        [0.89443128, 1.        ]]),
 array([[1.        , 0.88887314],
        [0.88887314, 1.        ]]),
 array([[1.        , 0.75204715],
        [0.75204715, 1.        ]]),
 array([[1.        , 0.80095072],
        [0.80095072, 1.        ]]))

In [152]:
peo_model_1.evaluate(ptk_con_x_data,ptk_con_y_data)
teo_model_1.evaluate(ptk_con_x_data,ptk_con_y_data)
keo_model_1.evaluate(ptk_con_x_data,ptk_con_y_data)
ptk_con_model_1.evaluate(ptk_con_x_data,ptk_con_y_data)
ptk_model_1.evaluate(ptk_x_data,ptk_y_data)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.7314 - mae: 0.6749
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.6863 - mae: 0.5287
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.8475 - mae: 0.6149
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.2442 - mae: 0.3596
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - loss: 0.1685 - mae: 0.3080


[0.1684652417898178, 0.3079543709754944]